In [52]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

from src import Spotify

spotify = Spotify()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# token

In [53]:
import json
import requests as req
import base64
import pprint

with open("auth.json") as json_file:
        json_data = json.load(json_file)

client_data = "{}:{}".format(
    json_data['CLIENT_ID'],
    json_data['CLIENT_SECRET']
)
authorization = "Basic " + \
    base64.b64encode(client_data.encode('ascii')).decode("ascii")

res = req.post("https://accounts.spotify.com/api/token",
               data={
                   "grant_type": 'client_credentials'
               },
               headers={
                   'Authorization': authorization
               })

token = res.json()['access_token']
pprint.pprint(res.json())

{'access_token': 'BQBr_CP7oHrPuBusj8UkAP0rkWlv4TglhVB1KeiL38qFKT7-Tg6pbE5jhxyKGi1XTKlNLCQ1NKWMKfwAwO8',
 'expires_in': 3600,
 'token_type': 'Bearer'}


# search

In [54]:
import requests as req
from urllib.parse import urlencode
import pandas as pd

search_uri = "https://api.spotify.com/v1/search"

query = urlencode({
    "q": "10cm 서랍",
    "type": "track",
    "market": "KR",
    "limit": 10,
    "offset": 0
})
headers = {
    "authorization": "Bearer {}".format(token)
}

res = req.get("{}?{}".format(search_uri, query), headers=headers)

In [55]:
spotify.search_tracks
spotify.sel_tracks

계속하시려면 next, 종료는 exit를 입력해주세요.exit


,id,name,artists,artists_name,image
0,2y4hHM6c48Qzk0bqh33XfB,Winter Sleep,3HqSLMAZ3g3d5poNaI7GOU,IU,https://i.scdn.co/image/ab67616d0000b2730d378c...
1,1OQ3PoRZRtE7RH0NIh3p1O,Drawer,6zn0ihyAApAYV51zpXxdEp,10cm,https://i.scdn.co/image/ab67616d0000b273b22853...


# artists

In [56]:
from functools import reduce

ids = reduce(
    lambda acc, cur: acc + cur + ",",spotify.sel_tracks['artists'].values, "")[:-1]

artists_uri = "https://api.spotify.com/v1/artists"
query = urlencode({
    "ids": ids
})
headers = {
    "authorization": "Bearer {}".format(token)
}

res = req.get("{}?{}".format(artists_uri, query), headers=headers)

In [57]:
spotify.get_genres
spotify.genres

,artists,genres
0,3HqSLMAZ3g3d5poNaI7GOU,"k-pop,pop"
1,6zn0ihyAApAYV51zpXxdEp,"k-pop,pop"


# audio features

In [60]:
ids = reduce(
    lambda acc, cur: acc + cur + ",",spotify.sel_tracks['id'].values, "")[:-1]

features_uri = "https://api.spotify.com/v1/audio-features"
query = urlencode({
    "ids": ids
})
headers = {
    "authorization": "Bearer {}".format(token)
}

res = req.get("{}?{}".format(features_uri, query), headers=headers)

In [61]:
spotify.get_features
spotify.features

,id,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,2y4hHM6c48Qzk0bqh33XfB,0.539,0.237,0,-8.452,0.0273,0.820,0,0.345,0.259,68.045
1,1OQ3PoRZRtE7RH0NIh3p1O,0.359,0.469,7,-8.060,0.0326,0.745,0,0.101,0.259,84.366


# recommendations

In [88]:
tracks = spotify.sel_tracks
genres = spotify.genres
features = spotify.features

In [89]:
merges = pd.merge(tracks, genres, on="artists")
merges = pd.merge(merges, features, on="id")

merges.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2 entries, 0 to 1
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                2 non-null      object 
 1   name              2 non-null      object 
 2   artists           2 non-null      object 
 3   artists_name      2 non-null      object 
 4   image             2 non-null      object 
 5   genres            2 non-null      object 
 6   danceability      2 non-null      float64
 7   energy            2 non-null      float64
 8   key               2 non-null      int64  
 9   loudness          2 non-null      float64
 10  speechiness       2 non-null      float64
 11  acousticness      2 non-null      float64
 12  instrumentalness  2 non-null      int64  
 13  liveness          2 non-null      float64
 14  valence           2 non-null      float64
 15  tempo             2 non-null      float64
dtypes: float64(8), int64(2), object(6)
memory usage:

In [90]:
seed_cols = ['id', 'artists', 'genres']
feature_cols = ['acousticness', 'danceability', 'energy', 
                'instrumentalness','key', 'liveness', 'loudness', 
                'speechiness', 'tempo', 'valence']

merges = merges[seed_cols + feature_cols]
for seed_col in seed_cols:
    if seed_col == "id":
        change_col = "seed_tracks"
    else:
        change_col = "seed_{}".format(seed_col)
    merges.rename({seed_col : change_col}, axis=1, inplace=True)
    
merges.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2 entries, 0 to 1
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   seed_tracks       2 non-null      object 
 1   seed_artists      2 non-null      object 
 2   seed_genres       2 non-null      object 
 3   acousticness      2 non-null      float64
 4   danceability      2 non-null      float64
 5   energy            2 non-null      float64
 6   instrumentalness  2 non-null      int64  
 7   key               2 non-null      int64  
 8   liveness          2 non-null      float64
 9   loudness          2 non-null      float64
 10  speechiness       2 non-null      float64
 11  tempo             2 non-null      float64
 12  valence           2 non-null      float64
dtypes: float64(8), int64(2), object(3)
memory usage: 224.0+ bytes


In [91]:
reco_uri = "https://api.spotify.com/v1/recommendations"

for idx in range(len(merges)):
    query = merges.iloc[idx].to_dict()
    query['market'] = 'KR'
    query['limit'] = 100
    
    query = urlencode(query)
    headers = {
        "authorization": "Bearer {}".format(token)
    }
    res = req.get("{}?{}".format(reco_uri, query), headers=headers)

In [94]:
spotify.get_reco_tracks
spotify.reco_tracks.head()

,id,name,artists,artists_name,image
0,1cayXr914ctuYeJkv3CPUR,Rest,6dhfy4ByARPJdPtMyrUYJK,Yerin Baek,https://i.scdn.co/image/ab67616d0000b273654022...
2,1e5qALs3pDrv203jX0XWAC,Autumn Breeze,0hRHbwZ0xSwfVHl4FTv7jq,Gummy,https://i.scdn.co/image/ab67616d0000b27339f1cc...
3,5AkyvofVWUqds8x1HHgDU9,Me After You,4qRXrzUmdy3p33lgvJEzdv,Paul Kim,https://i.scdn.co/image/ab67616d0000b27328ca61...
4,30aVEUz4NgkGuegz32gjl8,If you're with me,7jFUYMpMUBDL4JQtMZ5ilc,Sung Si Kyung,https://i.scdn.co/image/ab67616d0000b2736c6340...
5,5BqwC9kOBbqYkzdOKeXFFk,for lovers who hesitate,2SY6OktZyMLdOnscX3DCyS,JANNABI,https://i.scdn.co/image/ab67616d0000b273f694ad...
